# Introducción

En este documento se presenta la validacion del clasificador con el dataset UIEB

In [32]:

import os
import random
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense
import time
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
from sklearn.metrics import confusion_matrix
from torch.utils.data import RandomSampler
import pandas as pd
from PIL import ImageFile



In [33]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU no disponible, usando CPU")

Usando GPU: NVIDIA GeForce GTX 1060 6GB


# Listar Imagenes

In [3]:
def especifidad(MatrizConfusion):
    espeficifadM=(MatrizConfusion[1][1]/(MatrizConfusion[1][1]+MatrizConfusion[1][0]))*100
    return round(espeficifadM)

In [4]:
def recall(MatrizConfusion):
    recallM=(MatrizConfusion[0][0]/(MatrizConfusion[0][0]+MatrizConfusion[0][1]))*100
    return round(recallM)

In [5]:
def PrecisionClasePositiva(MatrizConfusion):
    PreP=(MatrizConfusion[0][0]/(MatrizConfusion[0][0]+MatrizConfusion[1][0]))*100
    return round(PreP)

In [6]:
def PrecisionClaseNegativa(MatrizConfusion):
    PreN=(MatrizConfusion[1][1]/(MatrizConfusion[1][1]+MatrizConfusion[0][1]))*100
    return round(PreN)

In [7]:
def exactitud(MatrizConfusion):
    exactitudM=((MatrizConfusion[0][0]+MatrizConfusion[1][1])/(MatrizConfusion[0][0]+MatrizConfusion[0][1]+MatrizConfusion[1][1]+MatrizConfusion[1][0]))*100
    return round(exactitudM)

# Descripcion de como Cargar un modelo

Se debe tener una instancia del modelo creada, para ello primero se tiene que definir el modelo. Con el siguiente codigo se define el modelo utilizado:

    def initialize_model2(num_classes):
        #Los pesos si se entrenan
        model_ft = models.inception_v3(weights=False) #Se carga el modelo inception_v3 sin sus pesos preentrenados.
        # False en las redes convencionales, se entrenaran
        set_parameter_requires_grad(model_ft, False)
        # Se modifica la red auxiliar
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)

        #Se modifica la red principal para que sea de dos clases con una salida softmax y coger la con mas probabilidad.
        model_ft.dropout= nn.Dropout(0.5)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(
            nn.Linear(num_ftrs,num_classes),
            nn.Dropout(0.3),
            nn.Softmax(dim=1)
        )

        input_size=299 #Tamaño que usa la red
        return model_ft,input_size
    


Y con la siguiente funcion:

    def set_parameter_requires_grad(model, feature_extracting):
        if feature_extracting:
            for param in model.parameters():
                param.requires_grad = False
            
            

Y a continuacion se instancia un modelo. Para hacer eso se deben utilizar las siguientes lineas de codigo:
Device cpu si no se dispone de cuda y si no  device = torch.device("cuda:0") con el indice deseado.

    device = torch.device("cpu")
    Modeloft,_= initialize_model2(2)
    Modelo=Modeloft.to(device)

    #Para inutilizar la capa auxiliar.
    Modeloft.aux_logits=False



Para poder cargar los pesos del modelo se hace con la siguiente linea:

    PATH = '..\Pesos\modeltranformaciones.pt'
    Modelo.load_state_dict(torch.load(PATH))
    


Y para finalizar para poder usarlo en modo prediccion hay que ponerlo en modo eval:

    Modelo.eval()



# Valores de test con el dataset UIEB

In [18]:
def initialize_model2(num_classes):
    #Los pesos si se entrenan
    model_ft = models.inception_v3(weights=False) #Se carga el modelo inception_v3 sin sus pesos preentrenados.
    # False en las redes convencionales, se entrenaran
    set_parameter_requires_grad(model_ft, False)
    # Se modifica la red auxiliar
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)

    #Se modifica la red principal para que sea de dos clases con una salida softmax y coger la con mas probabilidad.
    model_ft.dropout= nn.Dropout(0.5)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(
        nn.Linear(num_ftrs,num_classes),
        nn.Dropout(0.3),
        nn.Softmax(dim=1)
    )

    input_size=299 #Tamaño que usa la red
    return model_ft,input_size


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
        
        

device = torch.device("cpu")
Modeloft,_= initialize_model2(2)
Modelo=Modeloft.to(device)

#Para inutilizar la capa auxiliar.
Modeloft.aux_logits=False

PATH =  f'..\Pesos\Dataset 2 Modificado\modeltranformacionesEpoca38.pt'
Modelo.load_state_dict(torch.load(PATH))


Modelo.eval()

C:\Users\joseb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\joseb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [35]:
ValBuena_calidad2=os.listdir('..\Imagenes\Dataset 1\Validacion\Buenas-Reference/')
##ValBuena_calidad = list(filter(lambda x: x != 'Imagen', ValBuena_calidad))
ValBuena_calidad2= list(map(lambda  p: f"..\Imagenes\Dataset 1\Validacion\Buenas-Reference/{p}",ValBuena_calidad2))

ValMala_calidad2=os.listdir('..\Imagenes\Dataset 1\Validacion\Malas-Raw/')
##ValMala_calidad = list(filter(lambda x: x != 'Imagen', ValMala_calidad))
ValMala_calidad2= list(map(lambda  p: f"..\Imagenes\Dataset 1\Validacion\Malas-Raw/{p}",ValMala_calidad2))


ValBuena_calidad3=os.listdir('..\Imagenes\Dataset 1\Entreno\Buenas-Reference/')
##ValBuena_calidad = list(filter(lambda x: x != 'Imagen', ValBuena_calidad))
ValBuena_calidad3= list(map(lambda  p: f"..\Imagenes\Dataset 1\Entreno\Buenas-Reference/{p}",ValBuena_calidad3))

ValMala_calidad3=os.listdir('..\Imagenes\Dataset 1\Entreno\Malas-Raw/')
##ValMala_calidad = list(filter(lambda x: x != 'Imagen', ValMala_calidad))
ValMala_calidad3= list(map(lambda  p: f"..\Imagenes\Dataset 1\Entreno\Malas-Raw/{p}",ValMala_calidad3))


Val_Imagenesdat1=ValMala_calidad2+ValBuena_calidad2 + ValBuena_calidad3 + ValMala_calidad3
print("Imagenes de validacion totales", len(Val_Imagenesdat1))
random.shuffle(Val_Imagenesdat1)

val_dsdat1 = goodBadDataset2(Val_Imagenesdat1, transform)

Imagenes de validacion totales 1780


In [31]:
etiquetas_reales = np.zeros(len(val_dsdat1))
etiquetas_predichas = np.zeros(len(val_dsdat1))


for i in range(len(val_dsdat1)):
    img,target,path= val_dsdat1.__getitem__(i)
    etiquetas_reales[i]=target
    tensor_imagen = img.unsqueeze(0)
    with torch.no_grad():
        
        prediccion = Modelo(tensor_imagen)
        clase_mas_probable = torch.argmax(prediccion, dim=1)
        ##Prediccion[0][0] el % de buena calidad que tiene la imagen
        etiquetas_predichas[i]=clase_mas_probable.item()

# obtener la matriz de confusión
matriz_confusion = confusion_matrix(etiquetas_reales, etiquetas_predichas)

# imprimir la matriz de confusión
print("La matriz de confusion del Modelo X es: ")
print(matriz_confusion)

print("La precision de la clase de Buena calidad del Modelo X es de :",PrecisionClasePositiva(matriz_confusion),"%" )
print("El recall de la clase de Buena calidad del Modelo X es de :",recall(matriz_confusion),"%" )
print("La precision de la clase de Mala calidad del Modelo X es de :",PrecisionClaseNegativa(matriz_confusion),"%" )
print("La especifidad de la clase de Mala calidad del Modelo X es de :",especifidad(matriz_confusion),"%" )
print("La exactitud del Modelo X es de :",exactitud(matriz_confusion),"%" )

La matriz de confusion del Modelo X es: 
[[745 145]
 [180 710]]
La precision de la clase de Buena calidad del Modelo X es de : 81 %
El recall de la clase de Buena calidad del Modelo X es de : 84 %
La precision de la clase de Mala calidad del Modelo X es de : 83 %
La especifidad de la clase de Mala calidad del Modelo X es de : 80 %
La exactitud del Modelo X es de : 82 %
